## Dark Shower with PETITE

In [101]:
# Force reload (if changes have been made to the module)
%load_ext autoreload
%autoreload 2

import warnings
from platform import python_version
print("Python version: ", python_version())

import numpy
print("Numpy version: ", numpy.__version__)

import os
current_path = os.getcwd()
parent_dir = os.path.dirname(current_path)

PETITE_home_dir= parent_dir.split('examples')[0]

print("PETITE home directory:", PETITE_home_dir)

dictionary_dir = "/data_400GeV/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Python version:  3.11.3
Numpy version:  1.24.2
PETITE home directory: /Users/samuelpatrone/Desktop/Samuel/PhD/Papers/04 - SHIP ALPs/PETITE


In [15]:
from PETITE.dark_shower import *
from PETITE.shower import *
import pickle as pk
from matplotlib import pyplot as plt
from tqdm import tqdm
import matplotlib

from matplotlib.font_manager import FontProperties
from matplotlib.ticker import FixedLocator, MaxNLocator
import cProfile
profile = cProfile.Profile()
import pstats

font0 = FontProperties()
font = font0.copy()
font.set_size(24)
font.set_family('serif')
labelfont=font0.copy()
labelfont.set_size(20)
labelfont.set_weight('bold')
legfont=font0.copy()
legfont.set_size(18)
legfont.set_weight('bold')


def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)

In [54]:
malist=[0.05,0.1,0.2,0.4,0.6,0.8,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,2.1]
ma=malist[0]
sLead = DarkShower(PETITE_home_dir+dictionary_dir, "lead", 0.030, ma)

In [94]:
#Generate DarkVectors
vector_from_el_prim=[]
vector_from_el=[]
vector_from_pos=[]

me = 0.512*1e-6
n_electrons_on_target = 50
E0 = 10
for kk in tqdm(range(n_electrons_on_target)):
    p0 = [E0, 0, 0, np.sqrt(E0**2-me**2)]
    r0 = [0, 0, 0]
    pdict = {"PID":11, "weight":1.0/n_electrons_on_target}
    part0 = Particle(p0, r0, pdict)
    s0SM = sLead.generate_shower(part0)
    s0BSM = sLead.generate_dark_shower(ExDir=list(s0SM));
    for V in s0BSM[1]:   
        genprocess = V.get_ids()["generation_process"]
        parent_pid = V.get_ids()["parent_PID"]
        
        #DarkVector Stored infos
        p = V.get_p0()
        kyn_vars=V.get_ids()['kinematics_vars']
        w = V.get_ids()["weight"]
        parent_E = V.get_ids()["parent_E"]
        gen_number= V.get_ids()["generation_number"]
        
        
        DarkVector={'p': p, 'kinematics_vars':kyn_vars, 'weight': w, "parent_E": parent_E, 'gen_number': gen_number}
        
        if genprocess=="DarkBrem":
            if gen_number==1:
                if parent_pid==11: vector_from_el_prim.append(DarkVector)
            if parent_pid==11: vector_from_el.append(DarkVector)
            if parent_pid==-11: vector_from_pos.append(DarkVector)
                

100%|███████████████████████████████████████████| 50/50 [03:33<00:00,  4.27s/it]


## Differential Cross Section and Weights rescaling

In [112]:
def dsig_dx_dcostheta_dark_brem_exact_tree_level(x0, x1, x2, params):
    """Exact Tree-Level Dark Photon Bremsstrahlung  
       e (ep) + Z -> e (epp) + V (w) + Z
       result it dsigma/dx/dcostheta where x=E_darkphoton/E_beam and theta is angle between beam and dark photon

       Input parameters needed:
            x0, x1, x2:  kinematic parameters related to energy of emitted vector, cosine of its angle and the momentum transfer to the nucleus (precise relation depends on params['Method'] see below.
            me (mass of electron)
            mV (mass of dark photon)
            Ebeam (incident electron energy)
            ZTarget (Target charge)
            ATarget (Target Atomic mass number)  
            MTarget (Target mass)
    """
    me = params['me']
    mV = params['mV']
    Ebeam = params['E_inc']
    MTarget = params['mT']
    
    #SamADD_START
    GeV=1
    m_proton=938.272088*1e-3
    #SamADD_END
    
    if ('Method' in params.keys()) == False:
        params['Method'] = 'Log'
    if params['Method'] == 'Log':
        x, l1mct, lttilde = x0, x1, x2
        one_minus_costheta = 10**l1mct    
        costheta = 1.0 - one_minus_costheta
        ttilde = 10**lttilde
        Jacobian = one_minus_costheta*ttilde*np.log(10.0)**2
    elif params['Method'] == 'Standard':
        x, costheta, ttilde = x0, x1, x2
        Jacobian = 1.0

    # kinematic boundaries
    if x*Ebeam < mV:
        return 0.
    
    k = np.sqrt((x * Ebeam)**2 - mV**2)
    p = np.sqrt(Ebeam**2 - me**2)
    V = np.sqrt(p**2 + k**2 - 2*p*k*costheta)
    
    
    utilde = -2 * (x*Ebeam**2 - k*p*costheta) + mV**2
    
    discr = utilde**2 + 4*MTarget*utilde*((1-x)*Ebeam + MTarget) + 4*MTarget**2 * V**2
    # kinematic boundaries
    if discr < 0:
        return 0.
        
    Qplus = V * (utilde + 2*MTarget*((1-x)*Ebeam + MTarget)) + ((1-x)*Ebeam + MTarget) * np.sqrt(discr)
    Qplus = Qplus/(2*((1-x)*Ebeam + MTarget)**2-2*V**2)
    
    Qminus = V * (utilde + 2*MTarget*((1-x)*Ebeam + MTarget)) - ((1-x)*Ebeam + MTarget) * np.sqrt(discr)
    Qminus = Qminus/(2*((1-x)*Ebeam + MTarget)**2-2*V**2)
    
    Qplus = np.fabs(Qplus)
    Qminus = np.fabs(Qminus)
    
    tplus = 2*MTarget*(np.sqrt(MTarget**2 + Qplus**2) - MTarget)
    tminus = 2*MTarget*(np.sqrt(MTarget**2 + Qminus**2) - MTarget)

    # Physical region checks
    if tplus < tminus:
        return 0.
    
    tconv = (2*MTarget*(MTarget + Ebeam)*np.sqrt(Ebeam**2 + m_electron**2)/(MTarget*(MTarget+2*Ebeam) + m_electron**2))**2
    t = ttilde*tconv
    if t > tplus or t < tminus:
        return 0.
            
    q0 = -t/(2*MTarget)
    q = np.sqrt(t**2/(4*MTarget**2)+t)
    costhetaq = -(V**2 + q**2 + me**2 -(Ebeam + q0 -x*Ebeam)**2)/(2*V*q)

    # kinematic boundaries
    if np.fabs(costhetaq) > 1.:
        return 0.
    mVsq2mesq = (mV**2 + 2*me**2)
    Am2 = -8 * MTarget * (4*Ebeam**2 * MTarget - t*(2*Ebeam + MTarget)) * mVsq2mesq
    A1 = 8*MTarget**2/utilde
    Am1 = (8/utilde) * (MTarget**2 * (2*t*utilde + utilde**2 + 4*Ebeam**2 * (2*(x-1)*mVsq2mesq - t*((x-2)*x+2)) + 2*t*(-mV**2 + 2*me**2 + t)) - 2*Ebeam*MTarget*t*((1-x)*utilde + (x-2)*(mVsq2mesq + t)) + t**2*(utilde-mV**2))
    A0 = (8/utilde**2) * (MTarget**2 * (2*t*utilde + (t-4*Ebeam**2*(x-1)**2)*mVsq2mesq) + 2*Ebeam*MTarget*t*(utilde - (x-1)*mVsq2mesq))
    Y = -t + 2*q0*Ebeam - 2*q*p*(p - k*costheta)*costhetaq/V 
    W= Y**2 - 4*q**2 * p**2 * k**2 * (1 - costheta**2)*(1 - costhetaq**2)/V**2
    
    if W == 0.:
        print("x, costheta, t = ", [x, costheta, t])
        print("Y, q, p, k, costheta, costhetaq, V" ,[Y, q, p, k, costheta, costhetaq, V])
        
    # kinematic boundaries
    if W < 0:
        return 0.
    
    phi_integral = (A0 + Y*A1 + Am1/np.sqrt(W) + Y * Am2/W**1.5)/(8*MTarget**2)

    formfactor_separate_over_tsquared = Gelastic_inelastic_over_tsquared(params, t)
    
    ans = formfactor_separate_over_tsquared*np.power(alpha_em, 3) * k * Ebeam * phi_integral/(p*np.sqrt(k**2 + p**2 - 2*p*k*costheta))
    
    return(ans*tconv*Jacobian)

def dsig_dx_dcostheta_axion_brem_exact_tree_level(x0, x1, x2, params):
    """Exact Tree-Level Axion Photon Bremsstrahlung  
       e (ep) + Z -> e (epp) + a (w) + Z
       result it dsigma/dx/dcostheta where x=E_axion/E_beam and theta is angle between beam and axion

       Input parameters needed:
            x0, x1, x2:  kinematic parameters related to energy of emitted vector, cosine of its angle and the momentum transfer to the nucleus (precise relation depends on params['Method'] see below.
            me (mass of electron)
            ma (mass of axion)
            Ebeam (incident electron energy)
            ZTarget (Target charge)
            ATarget (Target Atomic mass number)  
            MTarget (Target mass)
    """
    me = params['me']
    ma = params['ma']
    Ebeam = params['E_inc']
    MTarget = params['mT']
    
    GeV=1
    m_proton=938.272088*1e-3

    if ('Method' in params.keys()) == False:
        params['Method'] = 'Log'
    if params['Method'] == 'Log':
        x, l1mct, lttilde = x0, x1, x2
        one_minus_costheta = 10**l1mct    
        costheta = 1.0 - one_minus_costheta
        ttilde = 10**lttilde
        Jacobian = one_minus_costheta*ttilde*np.log(10.0)**2
    elif params['Method'] == 'Standard':
        x, costheta, ttilde = x0, x1, x2
        Jacobian = 1.0

    # kinematic boundaries
    if x*Ebeam < ma:
        return 0.
    
    k = np.sqrt((x * Ebeam)**2 - ma**2)
    p = np.sqrt(Ebeam**2 - me**2)
    V = np.sqrt(p**2 + k**2 - 2*p*k*costheta)
    
    
    utilde = -2 * (x*Ebeam**2 - k*p*costheta) + ma**2
    
    discr = utilde**2 + 4*MTarget*utilde*((1-x)*Ebeam + MTarget) + 4*MTarget**2 * V**2
    # kinematic boundaries
    if discr < 0:
        return 0.
        
    Qplus = V * (utilde + 2*MTarget*((1-x)*Ebeam + MTarget)) + ((1-x)*Ebeam + MTarget) * np.sqrt(discr)
    Qplus = Qplus/(2*((1-x)*Ebeam + MTarget)**2-2*V**2)
    
    Qminus = V * (utilde + 2*MTarget*((1-x)*Ebeam + MTarget)) - ((1-x)*Ebeam + MTarget) * np.sqrt(discr)
    Qminus = Qminus/(2*((1-x)*Ebeam + MTarget)**2-2*V**2)
    
    Qplus = np.fabs(Qplus)
    Qminus = np.fabs(Qminus)
    
    tplus = 2*MTarget*(np.sqrt(MTarget**2 + Qplus**2) - MTarget)
    tminus = 2*MTarget*(np.sqrt(MTarget**2 + Qminus**2) - MTarget)

    # Physical region checks
    if tplus < tminus:
        return 0.
    
    tconv = (2*MTarget*(MTarget + Ebeam)*np.sqrt(Ebeam**2 + m_electron**2)/(MTarget*(MTarget+2*Ebeam) + m_electron**2))**2
    t = ttilde*tconv
    if t > tplus or t < tminus:
        return 0.
            
    q0 = -t/(2*MTarget)
    q = np.sqrt(t**2/(4*MTarget**2)+t)
    costhetaq = -(V**2 + q**2 + me**2 -(Ebeam + q0 -x*Ebeam)**2)/(2*V*q)

    # kinematic boundaries
    if np.fabs(costhetaq) > 1.:
        return 0.
    Am2 = 4 * MTarget * ma**2 * (-4 * Ebeam**2 * MTarget + 2 * Ebeam * t + MTarget * t)
    A1 = 4*MTarget**2/utilde
    Am1 = (4 * MTarget * (Ebeam**2 * (8 * MTarget * ma**2 * (x - 1) - 4 * MTarget * t * x**2) - 
                     2 * Ebeam * t * (ma**2 * (x - 2) + utilde * x) + 
                     MTarget * (2 * ma**2 * t + utilde**2))) / utilde
    A0 = (4 * MTarget * (-4 * Ebeam**2 * MTarget * ma**2 * (x - 1)**2 + 2 * Ebeam * t * (ma**2 - x * (ma**2 + utilde)) + MTarget * (ma**2 * t + 2 * utilde**2))) / utilde**2
    Y = -t + 2*q0*Ebeam - 2*q*p*(p - k*costheta)*costhetaq/V 
    W= Y**2 - 4*q**2 * p**2 * k**2 * (1 - costheta**2)*(1 - costhetaq**2)/V**2
    
    if W == 0.:
        print("x, costheta, t = ", [x, costheta, t])
        print("Y, q, p, k, costheta, costhetaq, V" ,[Y, q, p, k, costheta, costhetaq, V])
        
    # kinematic boundaries
    if W < 0:
        return 0.
    
    phi_integral = (A0 + Y*A1 + Am1/np.sqrt(W) + Y * Am2/W**1.5)/(8*MTarget**2)

    formfactor_separate_over_tsquared = Gelastic_inelastic_over_tsquared(params, t)
    
    ans = formfactor_separate_over_tsquared*np.power(alpha_em, 3) * k * Ebeam * phi_integral/(p*np.sqrt(k**2 + p**2 - 2*p*k*costheta))
    
    return(ans*tconv*Jacobian)

mu_p = 2.79  # https://journals.aps.org/prd/pdf/10.1103/PhysRevD.8.3109
def Gelastic_inelastic_over_tsquared(EI, t):
    """
    Form factor squared used for elastic/inelastic contributions to Dark Bremsstrahlung Calculation
    Rescaled by 1/t^2 to make it easier to integrate over t
    (Scales like Z^2 in the small-t limit)
    See Eq. (9) of Gninenko et al (Phys. Lett. B 782 (2018) 406-411)
    """
    Z = EI["Z_T"]
    A = EI["A_T"]
    c1 = (111 * Z ** (-1 / 3) / m_electron) ** 2
    c2 = 0.164 * GeV**2 * A ** (-2 / 3)
    Gel = (1.0 / (1.0 + c1 * t)) ** 2 * (1 + t / c2) ** (-2)
    ap2 = (773.0 * Z ** (-2.0 / 3) / m_electron) ** 2
    Ginel = (
        Z
        / ((c1**2 * Z**2))
        * np.power((ap2 / (1.0 + ap2 * t)), 2.0)
        * ((1.0 + (mu_p**2 - 1.0) * t / (4.0 * m_proton**2)) / (1.0 + t / 0.71) ** 4)
    )
    return Z**2 * c1**2 * (Gel + Ginel)


def rescale_weights(kinematics_vars, Einc, ma, weight, shower):

    #Build params dict
    ZTarget, ATarget, _, _ = shower.get_material_properties()
    params={'me': 0.512*1e-6, 'ma': ma, 'mV': ma, 'E_inc': Einc, 'mT': ATarget, "Z_T": ZTarget, "A_T": ATarget}
    x0=kinematics_vars[0]
    x1=kinematics_vars[1]
    x2=kinematics_vars[2]
    
    #Rescale weight
    new_weight=weight*dsig_dx_dcostheta_axion_brem_exact_tree_level(x0, x1, x2, params)/dsig_dx_dcostheta_dark_brem_exact_tree_level(x0, x1, x2, params)
    
    return new_weight

def convert_vec_to_axions(vector_array, ma, shower):
    axion_array=[]
    for vector in vector_array:
        generation = vector['gen_number'] 
        Einc = vector['parent_E'] 
        kyn_vars = vector['kinematics_vars'] 
        weight = vector['weight'] 
        rescaled_weight = rescale_weights(kyn_vars, Einc, ma, weight, shower)
        
        p=vector['p']
        mV=np.sqrt(p[0]**2-p[1]**2-p[2]**2-p[3]**2)
        if mV/ma-1>1e-10: warnings.warn("The mass of the vector and of the axion do not correspond!", UserWarning)
        
        axion_array.append({'p': p , 'kinematics_vars': kyn_vars, 'weight': rescaled_weight, "parent_E": Einc, 'gen_number': generation})
        
    return axion_array
        

In [118]:
ma=0.03
axion_from_el_prim=convert_vec_to_axions(vector_from_el_prim, ma, sLead)
axion_from_el=convert_vec_to_axions(vector_from_el, ma, sLead)
axion_from_pos=convert_vec_to_axions(vector_from_pos, ma, sLead)